## Code for ETL operations on Country-GDP data

In [104]:
# Importing the required libraries

import pandas as pd
from bs4 import BeautifulSoup
import requests
from forex_python.converter import CurrencyRates
import sqlite3
import numpy as np
import datetime

#### Task 1: Logging function


In [106]:
def log_progress(message):
    ''' This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing'''
    with open("code_log.txt", "a") as log_file:
        time_stamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_entry = f"{time_stamp} : {message}\n"
        log_file.write(log_entry)


# First log entry from the table

log_progress("Preliminaries complete. Initiating ETL process")


#### Task 2 : Extraction of data


In [108]:
def extract(url, table_attribs):
    # Send a GET request to the URL
    response = requests.get(url)
    
    # Raise an exception for unsuccessful HTTP requests
    response.raise_for_status()
    
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all tables on the page
    tables = soup.find_all('table')

    # Filter for the table with the specified attributes
    for table in tables:
        if all(attr in table.attrs.get('class', []) for attr in table_attribs['class']):
            # Extract column names from the table header
            headers = [header.text.strip() for header in table.find('tr').find_all('th')]

            # Extract data from the table and store it in a list of lists
            data = []
            for row in table.find_all('tr')[1:]:  # Skip the first row
                row_data = [cell.text.strip() for cell in row.find_all('td')]
                data.append(row_data)

            # Create a DataFrame from the extracted data
            df = pd.DataFrame(data, columns=headers)

            # Remove '\n' from the 'Market Cap' column and typecast to float
            if 'Market Cap' in df.columns:
                df['Market Cap'] = df['Market Cap'].str.rstrip('\n').astype(float)

            # Return the DataFrame
            return df

    # If no matching table is found
    print("Table not found on the webpage.")
    return pd.DataFrame()  # Return an empty DataFrame

# Loading Dataframe
url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
table_attributes = {'class': ['wikitable', 'sortable']}
result_df = extract(url, table_attributes)

# Display the resulting DataFrame
print(result_df)

# second log entry from the table

log_progress("Data extraction complete. Initiating Transformation process")


  Rank                                Bank name Market cap(US$ billion)
0    1                           JPMorgan Chase                  432.92
1    2                          Bank of America                  231.52
2    3  Industrial and Commercial Bank of China                  194.56
3    4               Agricultural Bank of China                  160.68
4    5                                HDFC Bank                  157.91
5    6                              Wells Fargo                  155.87
6    7                        HSBC Holdings PLC                  148.90
7    8                           Morgan Stanley                  140.83
8    9                  China Construction Bank                  139.82
9   10                            Bank of China                  136.81



#### Task 3


In [110]:
def transform(df, csv_path):
    # Step 1: Read exchange rate CSV file and create a dictionary
    exchange_rate_df = pd.read_csv(csv_path)
    exchange_rate = dict(zip(exchange_rate_df.iloc[:, 0], exchange_rate_df.iloc[:, 1]))

    # Check and convert the 'Market cap(US$ billion)' column to numeric if necessary
    df['Market cap(US$ billion)'] = pd.to_numeric(df['Market cap(US$ billion)'], errors='coerce')

    # Step 2: Add three new columns to the DataFrame using list comprehension
    for currency in ['GBP', 'EUR', 'INR']:
        new_column_name = f'MC_{currency}_Billion'
        df[new_column_name] = np.round([x * exchange_rate[currency] for x in df['Market cap(US$ billion)']],
                                       2)

    return df


# Provide the path to the exchange rate CSV file
exchange_rate_csv_path = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv'

# Apply the transformation to the DataFrame
result_df_transformed = transform(result_df, exchange_rate_csv_path)

# Display the resulting DataFrame with the new columns
print(result_df_transformed)

# third log entry from the table

log_progress("Data transformation complete. Initiating Loading process")


  Rank                                Bank name  Market cap(US$ billion)  \
0    1                           JPMorgan Chase                   432.92   
1    2                          Bank of America                   231.52   
2    3  Industrial and Commercial Bank of China                   194.56   
3    4               Agricultural Bank of China                   160.68   
4    5                                HDFC Bank                   157.91   
5    6                              Wells Fargo                   155.87   
6    7                        HSBC Holdings PLC                   148.90   
7    8                           Morgan Stanley                   140.83   
8    9                  China Construction Bank                   139.82   
9   10                            Bank of China                   136.81   

   MC_GBP_Billion  MC_EUR_Billion  MC_INR_Billion  
0          346.34          402.62        35910.71  
1          185.22          215.31        19204.58  
2      

#### Task 4


In [112]:
def load_to_csv(df, file_path):
    '''
    Save the DataFrame to a CSV file.

    Parameters:
    - df: pandas DataFrame
    - file_path: str, the path to save the CSV file
    '''
    try:
        df.to_csv(file_path, index=False)
        print(f"DataFrame successfully saved to {file_path}")
    except Exception as e:
        print(f"An error occurred while saving to CSV: {e}")


#csv_file_path = 'C:/Users/DELL/Desktop/Largest_banks_data.csv'
#load_to_csv(result_df_transformed, csv_file_path)
csv_file_path = 'Largest_banks_data.csv'
load_to_csv(result_df_transformed, csv_file_path)


# fourth log entry from the table

log_progress("Data saved to CSV file")

An error occurred while saving to CSV: [Errno 13] Permission denied: 'C:/Users/DELL/Desktop/output_transformed_data.csv'


#### Task 5


In [114]:
import sqlite3

def load_to_db(df, conn, table_name):
    '''
    Load the DataFrame into the specified table in the SQLite database.

    Parameters:
    - df: pandas DataFrame
    - conn: SQLite connection object
    - table_name: str, the name of the table in the database
    '''
    try:
        df.to_sql(table_name, conn, index=False, if_exists='replace')
        print(f"DataFrame successfully loaded into table {table_name} in the database.")
    except Exception as e:
        print(f"An error occurred while loading to the database: {e}")

# fifth log entry from the table

log_progress("SQL Connection initiated")


database_name = 'Banks.db'
table_name = 'Largest_Bank'  # Updated to 'Largest_Bank'
connection = sqlite3.connect(database_name)

# Apply the transformation to the DataFrame
result_df_transformed = transform(result_df, exchange_rate_csv_path)

# Display the resulting DataFrame with the new columns
print(result_df_transformed)

# Load the transformed DataFrame to the SQLite database
load_to_db(result_df_transformed, connection, table_name)


# sixth log entry from the table

log_progress("Data loaded to Database as a table, Executing queries")


  Rank                                Bank name  Market cap(US$ billion)  \
0    1                           JPMorgan Chase                   432.92   
1    2                          Bank of America                   231.52   
2    3  Industrial and Commercial Bank of China                   194.56   
3    4               Agricultural Bank of China                   160.68   
4    5                                HDFC Bank                   157.91   
5    6                              Wells Fargo                   155.87   
6    7                        HSBC Holdings PLC                   148.90   
7    8                           Morgan Stanley                   140.83   
8    9                  China Construction Bank                   139.82   
9   10                            Bank of China                   136.81   

   MC_GBP_Billion  MC_EUR_Billion  MC_INR_Billion  
0          346.34          402.62        35910.71  
1          185.22          215.31        19204.58  
2      


#### Task 6


In [116]:
import sqlite3

def run_queries(query, conn):
    '''
    Execute the provided query using the SQLite connection object.

    Parameters:
    - query: str, SQL query statement
    - conn: SQLite connection object
    '''
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        rows = cursor.fetchall()

        # Print the query statement
        print(f"Query: {query}")

        # Print the query output
        for row in rows:
            print(row)

    except sqlite3.Error as e:
        print(f"SQLite error: {e}")
    except Exception as e:
        print(f"An error occurred while executing the query: {e}")


database_name = 'Banks.db'
connection = sqlite3.connect(database_name)

# Query 1: Print the contents of the entire table
query1 = "SELECT * FROM Largest_Bank"
run_queries(query1, connection)
print()

# Query 2: Print the average market capitalization of all the banks in Billion USD
query2 = "SELECT AVG(MC_GBP_Billion) AS Average_Market_Cap_Billion_USD FROM Largest_bank"
run_queries(query2, connection)
print()

# Query 3: Print only the names of the top 5 banks
query3 = 'SELECT `Bank Name` from Largest_bank LIMIT 5'
run_queries(query3, connection)






# seventh log entry from the table

log_progress("Process Complete")


# Close the database connection
connection.close()


# seventh log entry from the table

log_progress("Server Connection closed")

Query: SELECT * FROM Largest_Bank
('1', 'JPMorgan Chase', 432.92, 346.34, 402.62, 35910.71)
('2', 'Bank of America', 231.52, 185.22, 215.31, 19204.58)
('3', 'Industrial and Commercial Bank of China', 194.56, 155.65, 180.94, 16138.75)
('4', 'Agricultural Bank of China', 160.68, 128.54, 149.43, 13328.41)
('5', 'HDFC Bank', 157.91, 126.33, 146.86, 13098.63)
('6', 'Wells Fargo', 155.87, 124.7, 144.96, 12929.42)
('7', 'HSBC Holdings PLC', 148.9, 119.12, 138.48, 12351.26)
('8', 'Morgan Stanley', 140.83, 112.66, 130.97, 11681.85)
('9', 'China Construction Bank', 139.82, 111.86, 130.03, 11598.07)
('10', 'Bank of China', 136.81, 109.45, 127.23, 11348.39)

Query: SELECT AVG(MC_GBP_Billion) AS Average_Market_Cap_Billion_USD FROM Largest_bank
(151.987,)

Query: SELECT `Bank Name` from Largest_bank LIMIT 5
('JPMorgan Chase',)
('Bank of America',)
('Industrial and Commercial Bank of China',)
('Agricultural Bank of China',)
('HDFC Bank',)
